In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [2]:
url = "https://www.ocasionplus.com/coches-ocasion?type%3DCAR%26sort%3DFECHAOP_DESC%26page%3D2"

res_coches = requests.get(url)
if res_coches.status_code == 200:
    print(res_coches.status_code)
else:
    print("None")

200


In [3]:
sopa_coches = BeautifulSoup(res_coches.content, "html.parser")
#print(sopa_coches.prettify())

In [4]:
#Cogemos el nombre
nombres_coches = sopa_coches.find_all("h2", {"style": "overflow-x:hidden"})
lista_nombres = []
for i in nombres_coches:
    lista_nombres.append(i.getText())
lista_nombres[:4]


['Skoda Kamiq 1.0 TSI Ambition  (110 CV)',
 'Peugeot 2008 SUV SUV BlueHDi 100 Signature (100 CV)',
 'Mercedes-Benz Clase CLA Shooting Brake (122 CV)',
 'Renault Grand Scenic Limited Blue dCi (120 CV) 7 PLAZAS']

In [11]:
#Cogemos los precios
precios = sopa_coches.find_all("div", {"class": "jsx-689670945 priceContainer"})

precio_original = []
precio_descuento = []

for precio in precios:
    print(precio)
    precio_texto = precio.text.split("€")
    
    if len(precio_texto) == 2:
        precio_original.append(float(precio_texto[0]))
        precio_descuento.append("Sin descuento")
    
    elif len(precio_texto) == 3: 
        precio_original.append(float(precio_texto[0]))  
        precio_descuento.append(float(precio_texto[1]))     

<div class="jsx-689670945 priceContainer"><span class="jsx-2870186023 priceLast">22.900<!-- -->€</span><span class="jsx-2689423372 price">20.818<!-- -->€</span></div>
<div class="jsx-689670945 priceContainer"><span class="jsx-2870186023 priceLast">14.200<!-- -->€</span><span class="jsx-2689423372 price">12.909<!-- -->€</span></div>
<div class="jsx-689670945 priceContainer"><span class="jsx-2870186023 priceLast">30.800<!-- -->€</span><span class="jsx-2689423372 price">28.000<!-- -->€</span></div>
<div class="jsx-689670945 priceContainer"><span class="jsx-2870186023 priceLast">23.900<!-- -->€</span><span class="jsx-2689423372 price">21.727<!-- -->€</span></div>
<div class="jsx-689670945 priceContainer"><span class="jsx-2870186023 priceLast">33.900<!-- -->€</span><span class="jsx-2689423372 price">30.818<!-- -->€</span></div>
<div class="jsx-689670945 priceContainer"><span class="jsx-2870186023 priceLast">16.850<!-- -->€</span><span class="jsx-2689423372 price">15.318<!-- -->€</span></div

In [15]:
len(precio_descuento)

20

In [16]:
km = sopa_coches.find_all("span", {"class": "jsx-3960002075 detail big"})
lista_km = []
for i in km:
    if "km" in i.getText():
        lista_km.append(i.getText())
len(lista_km)

20

In [19]:
diccionario_coche = {"Coche": lista_nombres, "PrecioRebajado": precio_descuento, "Precio Original": precio_original, "Kilómetros": lista_km}
diccionario_coche

{'Coche': ['Skoda Kamiq 1.0 TSI Ambition  (110 CV)',
  'Peugeot 2008 SUV SUV BlueHDi 100 Signature (100 CV)',
  'Mercedes-Benz Clase CLA Shooting Brake (122 CV)',
  'Renault Grand Scenic Limited Blue dCi (120 CV) 7 PLAZAS',
  'BMW Serie 5 BMW Serie 5 520d Touring 140 kW (190 CV',
  'Kia Stonic 1.0 T-GDi Eco-Dynamic Drive (120 CV)',
  'Mercedes-Benz Clase GLB MERCEDES GLB 2.0 200 D DCT (150CV) PAQUETE AMG',
  'Fiat 500X 1.3 MultiJet Mirror 4x2 (95 CV)',
  'Chevrolet Cruze 2.0 VCDI LS  (125 CV)',
  'SEAT Arona 1.0 TSI (115CV) FR Ecomotive',
  'Honda Civic 1.4 I-VTEC Comfort (100 CV)',
  'Toyota Yaris 1.0 City (69 CV)',
  'Citroen C3 PureTech 82 Feel (83 CV)',
  'SEAT Ibiza S&S Reference Plus (80 CV)',
  'Peugeot 308 SW SW BlueHDi 130 S&S Active Pack  (130 CV)',
  'Nissan X-Trail 2.0 dCi N-Connecta 4x2 XTronic Auto (177 CV)',
  'Citroen C5 Aircross 30 S&S Shine (131 CV)',
  'Ford Focus 1.6 TDCI Trend  (115 CV)',
  'Citroen C3 Aircross PureTech 110 S&S Feel (110 CV)',
  'Kia Soul 1.6 CRDi 

In [21]:
df = pd.DataFrame(diccionario_coche)
df

,Coche,PrecioRebajado,Precio Original,Kilómetros
0,Skoda Kamiq 1.0 TSI Ambition (110 CV),20.818,22.90,11.151 km
1,Peugeot 2008 SUV SUV BlueHDi 100 Signature (10...,12.909,14.20,52.846 km
2,Mercedes-Benz Clase CLA Shooting Brake (122 CV),28.0,30.80,56.126 km
3,Renault Grand Scenic Limited Blue dCi (120 CV)...,21.727,23.90,60.828 km
4,BMW Serie 5 BMW Serie 5 520d Touring 140 kW (1...,30.818,33.90,68.564 km
5,Kia Stonic 1.0 T-GDi Eco-Dynamic Drive (120 CV),15.318,16.85,36.395 km
6,Mercedes-Benz Clase GLB MERCEDES GLB 2.0 200 D...,39.0,42.90,58.565 km
7,Fiat 500X 1.3 MultiJet Mirror 4x2 (95 CV),14.136,15.55,80.787 km
8,Chevrolet Cruze 2.0 VCDI LS (125 CV),Sin descuento,5.30,162.006 km
9,SEAT Arona 1.0 TSI (115CV) FR Ecomotive,15.445,16.99,48.954 km
